In [ ]:
DATADIR = "/content/gdrive/My Drive/para" 
Categories = ["1on","50on","1arka","50arka","25on","25arka","background"] 
import os
import cv2
x = []
y = []
for category in Categories:
  path = os.path.join(DATADIR,category)
  print(path)
  os.chdir(path) 
  label = Categories.index(category)  #etiket
  for im in os.listdir(path):
      try:
          img = cv2.imread(im,cv2.IMREAD_COLOR)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          x.append(img)
          y.append(label)
      except Exception as e:
          pass

In [ ]:
height,width = x[0].shape[:2]
print(width,height)

import numpy as np
x = np.array(x)
y = np.array(y)
x = x.astype(np.float32) #to save space 
x = x/255#This ensures that all the values are between 0 and 1
print(len(x))
print("0",np.sum(y == 0)) 
print("1",np.sum(y == 1))
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15,shuffle=True)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.25,shuffle=True)
print(len(x_train))
print(len(x_val))
print(len(x_test))

In [ ]:
#one hot encoding
#0 [0 0]
#1 [0 1] ...
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)
y_val_one_hot = to_categorical(y_val)

print(y_train_one_hot)

height,width = x[0].shape[:2]
print(width,height)

In [ ]:
from keras.applications import InceptionV3
inception = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(width,height,3)
)

from keras import layers
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Sequential
from keras.optimizers import Adam
def build_model(backbone):
    model = Sequential() #katman katman model oluşturmak için
    model.add(backbone) #inception modelini ekleme
    model.add(layers.GlobalAveragePooling2D()) #çıktıları vektör haline getirme
    model.add(layers.Dropout(0.5)) #overfittingi engellemek için nöron elemesi
    model.add(layers.Dense(2, activation='softmax')) #sınıflandırma katmanı 

    model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model(inception)
model.summary()

In [ ]:
BATCH_SIZE = 16
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
        rotation_range = 90,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

In [ ]:
model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/models/coin3.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
history = model.fit_generator(
    train_generator.flow(x_train, y_train_one_hot, batch_size=BATCH_SIZE),
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=50,
    validation_data=(x_val, y_val_one_hot),
    callbacks=[ model_checkpoint]
)

In [ ]:
print(model.metrics_names)
print(model.evaluate(x_val, y_val_one_hot))
from sklearn import metrics
y_pred_one_hot = model.predict(x_val)
y_pred_labels = np.argmax(y_pred_one_hot, axis = 1)
y_true_labels = np.argmax(y_val_one_hot,axis=1)
confusion_matrix = metrics.confusion_matrix(y_true=y_true_labels, y_pred=y_pred_labels)
print(confusion_matrix)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
img = cv2.imread("/content/gdrive/My Drive/para/test_2.jpg",cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(gray, cmap = "gray")
plt.show()
all_circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT,0.9,120,param2 = 30,minRadius = 0, maxRadius = 200 )

In [ ]:
from google.colab.patches import cv2_imshow

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
if circles is not None:
    output = img.copy()
    circles = np.round(circles[0, :]).astype("int")
    print(circles)
    for (x, y, r) in circles:
        crop=img[(y - r):(y + r),(x - r):(x + r)]
        crop = cv2.resize(crop,(width, height))
        plt.imshow(crop)
        plt.show()

In [ ]:
y_pred_one_hot = model.predict(np.array([crop, ]))
print(y_pred_one_hot)
y_true_labels = np.argmax(y_pred_one_hot,axis=1)
index = y_true_labels[0]
print(Categories[index])